In [ ]:
# H2O_Quantum_Simulation_Final_v2.py

import numpy as np
import matplotlib.pyplot as plt

# Qiskit Nature imports
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

# Qiskit Aer for simulation
from qiskit_aer.primitives import Estimator as AerEstimator

# Qiskit Algorithms
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP

def run_h2o_simulation():
    """
    Performs a VQE simulation to find the ground state energy of a water molecule
    and plots the energy convergence.
    """
    print("Starting quantum simulation for the H₂O molecule...")

    # Define the Molecule
    h2o_geometry = "O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0"

    #Set up the Electronic Structure Problem
    driver = PySCFDriver(atom=h2o_geometry, basis="sto3g")
    problem = driver.run()

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    # Map the Problem to Qubits
    mapper = JordanWignerMapper()
    
    second_q_hamiltonian = problem.second_q_ops()[0]
    qubit_op = mapper.map(second_q_hamiltonian)

    #Step 4: Define VQE Componen
    init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    ansatz = UCCSD(num_spatial_orbitals, num_particles, mapper, initial_state=init_state)
    
    estimator = AerEstimator()
    optimizer = SLSQP(maxiter=100)

    # Step 5:Create a Callback to Monitor Convergence
    intermediate_data = {'iteration': [], 'energy': []}

    def vqe_callback(eval_count, parameters, mean, std):
        """Callback function to store intermediate results."""
        iteration = len(intermediate_data['iteration'])
        intermediate_data['iteration'].append(iteration)
        intermediate_data['energy'].append(mean)
        print(f"Iteration: {iteration:3d} | Energy: {mean:.12f}")

    #Step 6: Instantiate and Run VQE
    print("\nExecuting VQE algorithm...\n")
    initial_point = np.zeros(ansatz.num_parameters)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point, callback=vqe_callback)
    result = vqe.compute_minimum_eigenvalue(qubit_op)

    print("\nVQE Simulation Complete")
    print(f"Final Ground State Energy: {result.eigenvalue.real:.12f} Hartrees")

    #Step 7: Plot the Energy Convergence Graph
    print("\nGenerating convergence plot...")
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(intermediate_data['iteration'], intermediate_data['energy'], 'o-', color='dodgerblue', label='VQE Energy')
    ax.set_xlabel('Optimizer Iteration', fontsize=12)
    ax.set_ylabel('Energy (Hartree)', fontsize=12)
    ax.set_title('Convergence of H₂O Ground State Energy', fontsize=14)

    ax.axhline(y=result.eigenvalue.real, color='red', linestyle='--', label=f'Converged Value: {result.eigenvalue.real:.6f}')
    ax.legend(fontsize=12)

    plt.show()


if __name__ == "__main__":
    run_h2o_simulation()